# Ataque FGSM (Fast Gradient Sign Method)

### Seguridad en redes neuronales convolucionales
#### Autor: Jorge Calvo Martín

* Este ataque consiste en generar ejemplos adversarios, casi indistinguibles por el ojo humano, que sean capaces de confundir o engañar a un modelo de Deep Learning


### 1. ¿Qué es un Ataque Adversarial?

Un ataque adversarial consiste en realizar pequeñas modificaciones en una imagen (o cualquier otro dato de entrada) de forma que estas alteraciones, casi imperceptibles para el ojo humano, hagan que un modelo de clasificación cometa errores. Es decir, se busca "engañar" al modelo para que produzca una predicción incorrecta.

---

## 2. Principio del FGSM

La idea central del FGSM es encontrar una pequeña perturbación que, sumada a la imagen original, aumente la función de pérdida del modelo. De esta forma, el modelo se confunde y falla en su clasificación.

La fórmula para generar una imagen adversarial es:

$
\mathbf{X}_{adv} = \mathbf{X} + \epsilon \cdot \operatorname{sign} \Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big)
$

donde:
- $\mathbf{X}$ es la imagen original.
- \(y\) es la etiqueta correcta de la imagen.
- $J(\theta, \mathbf{X}, y)$ es la función de pérdida del modelo, con parámetros $\theta$.
- $\nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y)$ es el gradiente de la pérdida respecto a la imagen.
- $\operatorname{sign}(\cdot)$ toma el signo de cada componente del gradiente.
- $\epsilon$ es un pequeño factor que controla la magnitud de la perturbación.

**Interpretación:**  
El término $\operatorname{sign}\Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big)$ indica la dirección en la que cada píxel debe modificarse para incrementar la pérdida. Al multiplicar por $\epsilon$, aseguramos que la perturbación sea pequeña, y al sumarla a $\mathbf{X}$, obtenemos la imagen adversarial $\mathbf{X}_{adv}$.

---

## 3. Proceso Paso a Paso del Ataque FGSM

1. **Obtener la imagen original $\mathbf{X}$ y su etiqueta $y$:**  
   Se parte de la imagen que queremos atacar.

2. **Calcular la función de pérdida $J(\theta, \mathbf{X}, y)$:**  
   Se utiliza la función de pérdida del modelo, que compara la predicción con la etiqueta correcta.

3. **Calcular el gradiente:**  
   Se realiza backpropagation para obtener el gradiente de la pérdida respecto a la imagen:  
   $\nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y)$.

4. **Generar la perturbación:**  
   Se obtiene la dirección de cambio mediante $\operatorname{sign}\Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big)$ y se multiplica por $\epsilon$.

5. **Crear la imagen adversarial:**  
   La imagen modificada se calcula como:  
   $\mathbf{X}_{adv} = \mathbf{X} + \epsilon \cdot \operatorname{sign}\Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big)$.

6. **Evaluar el modelo con $\mathbf{X}_{adv}$:**  
   Al introducir la imagen adversarial en el modelo, se observa que la predicción cambia, generalmente de forma incorrecta.

---

## 4. Ejemplo Matemático Sencillo

Supongamos que tenemos una imagen representada como un vector $\mathbf{X} = [x_1, x_2, \ldots, x_n]$ y queremos atacar esta imagen. Imaginemos que para un modelo concreto, el gradiente de la pérdida con respecto a la imagen es:

$$
\nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) = [0.2, -0.4, 0.1, \ldots, 0.3]
$$

Aplicamos la función de signo a este gradiente:

$$
\operatorname{sign} \Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big) = [1, -1, 1, \ldots, 1]
$$

Si elegimos un valor pequeño, por ejemplo, $\epsilon = 0.01$, la perturbación que añadiremos será:

$$
\epsilon \cdot \operatorname{sign} \Big( \nabla_{\mathbf{X}} J(\theta, \mathbf{X}, y) \Big) = [0.01, -0.01, 0.01, \ldots, 0.01]
$$

La imagen adversarial se obtiene sumando esta perturbación a la imagen original:

$$
\mathbf{X}_{adv} = \mathbf{X} + [0.01, -0.01, 0.01, \ldots, 0.01]
$$

Incluso si esta suma apenas modifica los valores originales, la dirección de la perturbación puede hacer que el modelo confunda la imagen y realice una clasificación errónea.

---

## 5. Conclusión

El método FGSM demuestra cómo, mediante pequeños cambios calculados en la entrada, se puede engañar a un modelo de clasificación. A pesar de que la perturbación es casi imperceptible para el ojo humano, puede ser suficiente para que el modelo cometa un error, lo que subraya la importancia de desarrollar sistemas robustos frente a ataques adversariales.

Este ejemplo y explicación deberían ayudarte a entender de forma práctica cómo se realiza un ataque adversarial utilizando FGSM.

---

*Puedes incluir este contenido en una celda Markdown de tu notebook para tener la explicación completa y el ejemplo matemático a la vista.*


In [ ]:
from IPython.display import display, Math, Latex
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from keras.preprocessing import image

Cuando entrenamos un modelo tenemos que pasarle unas entradas, que normalmente se encuentran acotadas dentro de un dominio. Por ejemplo, los píxeles de las imágenes toman valores enteros entre 0 y 255. Por tanto, nuestra red se debería comportar de la misma forma si introducimos los píxeles con intensidades X = [21, 34, 128, 11] y Xs = [21.001, 34.001, 128.001, 11.001]. La realidad es que no es así. Como vimos antes en una red sencilla se realiza la operación de la Figura 3. Eliminaremos algunos términos para facilitar la explicación, dejando la expresión de la siguiente forma: 

In [ ]:
display(Math(r'\hat{Y} = X * W'))

Como vemos nuestra entrada Xs es el resultado de sumar X + S, siendo S = [0.001, 0.001, 0.001, 0.001]. Por tanto, si introducimos como entrada Xs la salida vendrá determinada por la siguiente ecuación: 

In [ ]:
display(Math(r'\hat{Y_S} = X * W + S *W'))

Con esta ecuación podemos ver que si W es positivo y su dimensión es muy grande (recordad que en casos reales estamos hablando de millones de parámetros) podríamos producir un cambio muy grande en la salida haciendo un cambio muy pequeño en la entrada. La amplificación de la salida viene dada por la fórmula E*N*M, siendo E = ||S||, N el número de dimensiones de W y M el valor medio de W. El ataque FSGM modifica la imagen de entrada siguiendo la fórmula siguiente:

In [ ]:
display(Math(r'X_S = X + S'))
display(Math(r'S = \epsilon * sign(\nabla J(X,Y))'))
display(Math(r'J = Función\ de\ coste'))

Como podéis ver estamos calculando el signo de los gradientes de la función de coste con respecto a la imagen de entrada y después lo estamos multiplicando por un valor E. El signo de los gradientes nos proporciona la dirección aproximada de maximización de la función de coste y con E regulamos la cantidad de pérdida. Por tanto, estamos modificando la imagen de forma que maximicemos la probabilidad de que la red falle. Sé que puede sonar un poco complicado, por ello vamos a intentar verlo gráficamente. 

### Predecir una foto usando el modelo MobilNet

In [ ]:
mobilNet=tf.keras.applications.MobileNetV2(include_top=True,weights='imagenet')

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
def process_image(img):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = img[None, ...]
    return img

In [ ]:
def predecir_Net(img):
    #Cargamos la foto con un tamaño ya establecido que nos pide el modelo (224,224)
  
    img=process_image(img)
    
    #Se lo pasamos al modelo
    y=mobilNet.predict(img)
    
    #print(y.shape)
    #devuelve un valor por cada clase que predice (1000 clases), el llamado one-hot-encoding
    
    #Utilizamos decode_predicctions para asociar el valor a cada clase
    return img,y,decode_predictions(y, top=1)[0][0]

In [ ]:
image_raw = tf.io.read_file("examples/beer.jpg")
image = tf.image.decode_image(image_raw)
img,y,prediction=predecir_Net(image)
#img.shape

In [ ]:
#Saco la posición maxima del vector de predicción, el cual me dará el número de la clase.
max_index_row = np.argmax(y, axis=1)
max_index_row[0]

In [ ]:
plt.figure()
plt.imshow(process_image(image)[0]*0.5+0.5)
plt.title('Clase: {} {} : {:.2f}% Eficacia'.format(max_index_row[0],prediction[1], prediction[2]*100))
plt.show

In [ ]:
#convierto las etiquestas en un vector  One hot encoding
label=tf.one_hot(max_index_row,y.shape[-1])
label = tf.reshape(label, (1, y.shape[-1]))

### Generar Imagen Adversaria usando FGSM

In [ ]:
print(process_image(image).shape)
#Generamos la función de coste
loss_function=tf.keras.losses.CategoricalCrossentropy()

In [ ]:
#Creamos la imagen adversaria
def create_adversary(x,y):
    with tf.GradientTape() as tape:
        tape.watch(x)
        y_hat = mobilNet(x)
        loss = loss_function(y, y_hat)
        #print(loss)

      # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, x)
      # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
process_image(image).shape

In [ ]:
signed_grad = create_adversary(process_image(image),label)
plt.imshow(np.reshape(signed_grad,(224,224,3)))
plt.show()

In [ ]:
#Aplicamos la formula
display(Math(r'adv_{img}= X + \epsilon * sign(\nabla J(X,Y))'))
adv_img = process_image(image) + 0.01*signed_grad

In [ ]:
y=mobilNet.predict(adv_img)
prediction_adv=decode_predictions(y, top=1)[0][0]

In [ ]:
plt.figure()
plt.imshow(adv_img[0]*0.5+0.5)
plt.title('{} : {:.2f}% Eficacia'.format(prediction_adv[1], prediction_adv[2]*100))
plt.show